## Exploratory data analysis

Read in and analyse training data, then save important information to JSON file for later use in feature engineering and model training.

In [124]:
# Imports

import pandas as pd
import copy

In [125]:
# read csv into dataframe
data = pd.read_csv('../data/raw/train.csv')
data.shape

test = pd.read_csv('../data/raw/test.csv')

In [126]:
print(data.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [127]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [128]:
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [129]:
info = {}
info['columns'] = data.columns
info['data_types'] = data.dtypes 

In [130]:
# Check for missing values
missing_values = data.isnull().sum()
print(missing_values)

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


In [131]:
info['missing_values'] = copy.deepcopy(missing_values)
rows_with_missing_values = data[data.isnull().any(axis=1)]
info['rows_with_missing_values'] = rows_with_missing_values
rows_with_missing_values.shape

(1460, 81)

In [132]:
# Number of cols with missing values
missing_values = data.isnull().sum()
missing_values = missing_values[missing_values > 0]
num_cols_missing_vals = missing_values.shape[0]
info['num_cols_missing_vals'] = num_cols_missing_vals
num_cols_missing_vals

19

### Data Editing

In [133]:
# Convert all non-numerical data to categorical data
# Select non-numerical columns
non_numerical_cols = data.select_dtypes(exclude=['int64', 'float64']).columns
test_non_numerical_cols = test.select_dtypes(exclude=['int64', 'float64']).columns

# number of non-numerical columns
print(len(non_numerical_cols))
print(len(test_non_numerical_cols))

print(data.shape)
print(test.shape)

# Convert non-numerical columns to one-hot encoding
one_hot_data = pd.get_dummies(data, columns=non_numerical_cols)
one_hot_test = pd.get_dummies(test, columns=test_non_numerical_cols)

# get column names in one_hot_data that are not in one_hot_test
cols_not_in_test = one_hot_data.columns.difference(one_hot_test.columns)
print(cols_not_in_test)

# add clumns to one_hot_test with default value of 0 except for SalePrice
for col in cols_not_in_test:
    if col == 'SalePrice':
        pass
    else:
        one_hot_test[col] = 0

print(one_hot_data.shape)
print(one_hot_test.shape)
# save to csv
one_hot_data.to_csv('../data/processed/train_one-hot-encoded.csv', index=False)
one_hot_test.to_csv('../data/processed/test_one-hot-encoded.csv', index=False)

43
43
(1460, 81)
(1459, 80)
Index(['Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn',
       'Electrical_Mix', 'Exterior1st_ImStucc', 'Exterior1st_Stone',
       'Exterior2nd_Other', 'GarageQual_Ex', 'Heating_Floor', 'Heating_OthW',
       'HouseStyle_2.5Fin', 'MiscFeature_TenC', 'PoolQC_Fa',
       'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal',
       'RoofMatl_Roll', 'SalePrice', 'Utilities_NoSeWa'],
      dtype='object')
(1460, 289)
(1459, 288)


In [134]:
# Check for missing values
missing_values = one_hot_data.isnull().sum()
cols_with_missing_values = missing_values[missing_values > 0]
print(cols_with_missing_values)

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64


In [141]:
one_hot_copy = copy.deepcopy(one_hot_data)
one_hot_test_copy = copy.deepcopy(one_hot_test)
# Remove cols with missing values 
one_hot_copy = one_hot_copy.drop(cols_with_missing_values.index, axis=1)
one_hot_test_copy = one_hot_test_copy.drop(cols_with_missing_values.index, axis=1)

# Get the order of columns in one_hot_copy (excluding 'SalePrice') and rearrange one_hot_test_copy to match
cols = one_hot_copy.columns
cols = cols.drop('SalePrice')
one_hot_test_copy = one_hot_test_copy[cols]

# save to csv
one_hot_copy.to_csv('../data/processed/train_one-hot-encoded_no-missing_vals_cols_removed.csv', index=False)
one_hot_test_copy.to_csv('../data/processed/test_one-hot-encoded_no-missing_vals_cols_removed.csv', index=False)
print(one_hot_copy.shape)
print(one_hot_test_copy.shape)

(1460, 286)
(1459, 285)


In [144]:
# Check for nan values
print(one_hot_copy.isnull().values.any())
print(one_hot_test_copy.isnull().values.any())

False
True


In [145]:
# get cols in test where nan values are present
missing_values = one_hot_test_copy.isnull().sum()
cols_with_missing_values = missing_values[missing_values > 0]
print(cols_with_missing_values)

BsmtFinSF1      1
BsmtFinSF2      1
BsmtUnfSF       1
TotalBsmtSF     1
BsmtFullBath    2
BsmtHalfBath    2
GarageCars      1
GarageArea      1
dtype: int64


In [136]:
one_hot_copy = one_hot_data.fillna(one_hot_data.mean())
one_hot_test_copy = one_hot_test.fillna(one_hot_test.mean())
# save to csv
one_hot_copy.to_csv('../data/processed/train_one-hot-encoded_no-missing_vals_mean_imputed.csv', index=False)
one_hot_test_copy.to_csv('../data/processed/test_one-hot-encoded_no-missing_vals_mean_imputed.csv', index=False)

In [137]:
# Create copy of the dataset which removes columns with missing values
data_copy = data.dropna(axis=1)
test_copy = test.dropna(axis=1)
# save the data to csv
data_copy.to_csv('../data/processed/train_no_missing_vals_cols_removed.csv', index=False)
test_copy.to_csv('../data/processed/test_no_missing_vals_cols_removed.csv', index=False)
data_copy.shape

(1460, 62)

### Visualisation

In [138]:
#  Use histograms and box plots to understand distributions and identify outliers of numerical features
import matplotlib.pyplot as plt
import seaborn as sns

# Histograms
# Create a histogram for each numerical feature
# for col in data_copy.select_dtypes(include=['int64', 'float64']).columns:
#     plt.hist(data_copy[col])
#     plt.title(col)
#     plt.show()